In [3]:
!pip install gspread
!pip install cfbd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.9/186.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 22.2 MB/s eta 0:00:00


In [21]:
import cfbd
from cfbd.rest import ApiException
import gspread
import pandas as pd
import json

In [2]:
cbd_api_key = 'YOUR-API-KEY-HERE'

# Configure API key authorization: ApiKeyAuth
configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = cbd_api_key
configuration.api_key_prefix['Authorization'] = 'Bearer'

games_api_instance = cfbd.GamesApi(cfbd.ApiClient(configuration))

In [39]:
game_results = games_api_instance.get_games(year = 2023, season_type = 'regular')

game_rows = []

for game_dict in game_results:

    append_dict = {

            'attendance': game_dict.attendance,
            'away_conference': game_dict.away_conference,
            'away_division': game_dict.away_division,
            'away_id': game_dict.away_id,
            'away_points': game_dict.away_points,
            'away_post_win_prob': game_dict.away_post_win_prob,
            'away_postgame_elo': game_dict.away_postgame_elo,
            'away_pregame_elo': game_dict.away_pregame_elo,
            'away_team': game_dict.away_team,
            'conference_game': game_dict.conference_game,
            'excitement_index': game_dict.excitement_index,
            'home_conference': game_dict.home_conference,
            'home_division': game_dict.home_division,
            'home_id': game_dict.home_id,
            'home_points': game_dict.home_points,
            'home_post_win_prob': game_dict.home_post_win_prob,
            'home_postgame_elo': game_dict.home_postgame_elo,
            'home_pregame_elo': game_dict.home_pregame_elo,
            'home_team': game_dict.home_team,
            'game_id': game_dict.id,
            'neutral_site': game_dict.neutral_site,
            'season': game_dict.season,
            'season_type': game_dict.season_type,
            'start_date': game_dict.start_date,
            'start_time_tbd': game_dict.start_time_tbd,
            'venue_name': game_dict.venue,
            'venue_id': game_dict.venue_id,
            'game_week': game_dict.week,
        }


    away_period_points_array = game_dict.away_line_scores
    home_period_points_array = game_dict.home_line_scores

    if away_period_points_array is not None:
        for period_index in range(0, 13):

            period_num = period_index + 1

            if period_index < len(away_period_points_array):
                away_period_score = away_period_points_array[period_index]
            else:
                away_period_score = None

            if period_index < len(home_period_points_array):
                home_period_score = home_period_points_array[period_index]
            else:
                home_period_score = None


            append_dict['away_period_' + str(period_num)] = away_period_score
            append_dict['home_period_' + str(period_num)] = home_period_score
    else:
        for period_index in range(0, 13):
            
            period_num = period_index + 1

            append_dict['away_period_' + str(period_num)] = None
            append_dict['home_period_' + str(period_num)] = None

    game_rows.append(append_dict)

game_df = pd.DataFrame(game_rows)

game_df.fillna('', inplace=True)

In [40]:
gc = gspread.service_account(filename = 'PATH-TO-GOOGLE-CREDENTIALS.json')

cfbd_spreadsheet = gc.open('cfbd_games')
cfbd_games_sheet = cfbd_spreadsheet.worksheet('Sheet1')

cfbd_games_sheet.update([game_df.columns.values.tolist()] + game_df.values.tolist())

{'spreadsheetId': '1-sSvDQ0f_WSbEXpmajf8VeBUMjp_4fFxGH0ogkT1q3s',
 'updatedRange': 'Sheet1!A1:BB3600',
 'updatedRows': 3600,
 'updatedColumns': 54,
 'updatedCells': 194400}